### ▶️ Now lets import some python-libraries

In [19]:
import pandas as pd
import datetime
from IPython.display import display, HTML

### ▶️ Reading the prepared data from Google-Drive

#### This takes about 50 !! seconds to load... 
---

- Dataset includes a time period from 20-01-2014 till 20-01-2024
- It contents measurements for "Mittlere Temperatur", "Sonnenscheindauer" and "Windgeschwindigkeit"
- To made this a real world project with big data, it includes ALL german weather stations
- In summary the .csv-file contains over 3 Million datapoints and has a size of nearly 200 MegaBytes

In [2]:
# Import the .csv-file with the weather datasets
df = pd.read_csv('https://drive.usercontent.google.com/download?id=11DYEiMTRVhMySsH5r3cDfCWBU2mqlNqO&export=download&authuser=0&confirm=t&uuid=1aa8951f-3209-4a18-986c-229dfc0f29e2&at=APZUnTVUwg__SVFWwQUEz9ft2zsy:1705864211167', index_col=False)

df.head()

,Zeitstempel,Wert,Kategorie,Station
0,2014-01-20,-1.2,Mittlere Temperatur,Doberlug-Kirchhain
1,2014-01-21,-2.0,Mittlere Temperatur,Doberlug-Kirchhain
2,2014-01-22,-4.4,Mittlere Temperatur,Doberlug-Kirchhain
3,2014-01-23,-4.5,Mittlere Temperatur,Doberlug-Kirchhain
4,2014-01-24,-5.3,Mittlere Temperatur,Doberlug-Kirchhain


### ▶️ We have to convert our datatypes first

In [3]:

# Convert "Zeitstempel" to dateTime
df['Zeitstempel'] = pd.to_datetime(df.Zeitstempel)

# Convert "Station" to String
df['Station'] = df['Station'].astype("string")

# Convert "Produkt_Titel" to String
df['Kategorie'] = df['Kategorie'].astype("string")


### Some filters

In [ ]:
# Filter by single day
#df_filtered = df1[df1['Zeitstempel'].dt.strftime('%Y-%m-%d') == '2020-01-01']

# Filter by single month
#df_filtered = df1[df1['Zeitstempel'].dt.strftime('%Y-%m') == '2020-01']

# Filter by single month
#df_filtered = df.loc[(df['Zeitstempel'].dt.strftime('%Y') == '2022') & (['Station'] == "Alsfeld")]

#df_filtered = df.loc[((df['Kategorie'] == "Mittlere Temperatur") & (df['Station'] == "Doberlug-Kirchhain") & (df['Zeitstempel'].dt.strftime('%Y') == '2022'))]

#df_filtered = df.loc[((df['Kategorie'] == "Sonnenscheindauer") & (df['Station'] == "Doberlug-Kirchhain") & (df['Zeitstempel'].dt.strftime('%Y') == '2017'))]


### ▶️ Now we can start to filter our data

In [238]:
pd.set_option('display.max_colwidth', None)

def display_side_by_side(dfs:list, captions:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0\xa0\xa0"
    display(HTML(output))

# We take the data for "Sonnenscheindauer"
df_sunshine = df.loc[(df['Kategorie'] == "Sonnenscheindauer")].reset_index(drop=True)
df_sunshine_2 = df_sunshine.groupby('Station')

# We group our pre-chosen "Sonnenscheindauer" data by station, then we aggregate Minimum, Maximum, Mean and Variance

# Sub-set is finally sorted by Minimum in a ascending manner
df_sunshine_min = df_sunshine.groupby('Station')[['Wert']].agg(['min']).sort_values(by=('Wert','min'),ascending=True).reset_index()

# Sub-set is finally sorted by Maximum in a ascending manner
df_sunshine_max = df_sunshine.groupby('Station')[['Wert']].agg(['max']).sort_values(by=('Wert','max'),ascending=False).reset_index()

# Sub-result is finally sorted by Maximum in a ascending manner
df_sunshine_var = df_sunshine.groupby('Station')[['Wert']].agg(['var']).sort_values(by=('Wert','var'),ascending=True).reset_index()

# Sub-result is finally sorted by Mean in a ascending manner
df_sunshine_mean = df_sunshine.groupby('Station')[['Wert']].agg(['mean']).sort_values(by=('Wert','mean'),ascending=False).reset_index()
#df_sunshine_mean = df_sunshine_mean.loc[(df['Wert'] >= 2.0)]

# We resolve the mulitindex to a single index, and give the columns a nicer name
df_sunshine_max.columns = ['Station', 'Maximalwerte']
df_sunshine_min.columns = ['Station', 'Minimalwerte']
df_sunshine_mean.columns = ['Station', 'Mittelwerte']
df_sunshine_var.columns = ['Station', 'Varianzen']

display_side_by_side([df_sunshine_min[0:100], df_sunshine_max[0:100], df_sunshine_mean[0:100], df_sunshine_var[0:100]], ['Minima absteigend', 'Maxima absteigend', 'Mittelwerte absteigend', 'Varianzen aufsteigend'])

,Station,Minimalwerte
0,Aachen-Orsbach,0.000000
1,Neustadt am Kulm-Filchendorf,0.000000
2,Neuruppin,0.000000
3,Neunkirchen-Seelscheid-Krawinkel,0.000000
4,Neuhütten/Spessart,0.000000
5,Neuhaus am Rennweg,0.000000
6,"Neuenahr, Bad-Ahrweiler",0.000000
7,Neubulach-Oberhaugstett,0.000000
8,Neu-Ulrichstein,0.000000
9,Naumburg/Saale-Kreipitzsch,0.000000
